# Creating the Graphs

In [ ]:
from preprocessing.objects import create_objects
from preprocessing.author import Author
import networkit as nk
from networkit.vizbridges import Dimension

NetworKit is organized in **modules**.

`nk.<module_name>.<feature_name>`

## 1. Importing the Data

In [ ]:
author_list, publication_list = create_objects('Catalogdatabase-till2018b.xlsx', n=20)

## 2. Creating the Graph

In [ ]:
G = nk.Graph(weighted=True)
print(G.numberOfNodes(), G.numberOfEdges())
print(G.isWeighted(), G.isDirected())

In [ ]:
%time
author_list[0].id = G.addNode()
for index_a in range(len(author_list)):
    for index_b in range(index_a + 1, len(author_list)):
        author_a = author_list[index_a]
        author_b = author_list[index_b]
        author_b.id = G.addNode()

        common_publications = list(set(author_a.publications).intersection(author_b.publications))
        if common_publications:
            # Add the weighted edge to the graph
            G.addEdge(author_a.id, author_b.id, w=len(common_publications))

In [ ]:
# not even a little performant
#nk.vizbridges.widgetFromGraph(G, Dimension.Two)

# 2. Visualizing a graph with Gephi

In [ ]:
# Crete a streaming client for Gephi
client = nk.gephi.streaming.GephiStreamingClient("http://localhost:8080/workspacege1")

# Export the graph
client.exportGraph(G)

# 3 Computation of Central Vertices

## 3.1 Harmonic Closeness Centrality

Let $G = (V, E)$ be our input graph. The **harmonic closeness** of a vertex $v \in V$ is defined as:

$$
h(v) = \sum_{w \in V\setminus\{v\}}\frac{n - 1}{d(v, w)}
$$

- Measures how much a vertex $v$ is "close" to the other vertices
- Time complexity (per vertex): $\mathcal{O}(n + m)$ for unweighted

### Compute the harmonic closeness of all the vertices in $G$<br/>Visualize the distribution of the scores

In [ ]:
# General NetworKit algorithm pipeline

# 1. Create an intance of the algorithm
hc = nk.centrality.HarmonicCloseness(G, normalized=False)

# 2. Run the algorithm
hc.run()

# 3. Extract the results
harmonic = hc.scores()

# 4. Visualize the data
_=plt.hist(harmonic)

In [ ]:
# Export harmonic centrality values to Gephi
client.exportNodeValues(G, harmonic, 'harmonic_c')

## Vertex ranking

Often we are interested in the ranking of the top-$k$ most central vertices.

In [ ]:
# Set a value for k
k = 5

# Get the top-5 ranking
top_k = hc.ranking()[:k]

# Print the top-k ranking
for i, (vertex, score) in zip(range(k), top_k):
    print("Top-{:d} vertex: {} -- score: {:.2f}".format(i + 1, vertex, score))

In [ ]:
# Label vertices:
# True => is in top-5
# False => is not in top-5

in_top_k = [False for _ in G.iterNodes()]

for top_vertex, _ in top_k:
    in_top_k[top_vertex] = True

# Export labels to Gephi
client.exportNodeValues(G, in_top_k, "in_top_k")

### Further material not covered here: efficient top-$k$ vertex ranking

# 4. Graph Generators

The **generators** module provides a wide range of models to generate graphs.

` G = nk.generators.<generator_constructor>.generate()`

### 4.1 Random Hyperbolic Generator

Generates **complex networks** with a power-law degree distribution. Parameters:

`n` number of vertices in the graph

`avg_deg` average vertex degree

`gamma` exponent of the power-law degree distribution

In [ ]:
# Parameters:
n, avg_deg, exp = 100000, 10, 3

# Gererate the graph
G_cplx = nk.generators.HyperbolicGenerator(n, avg_deg, exp).generate()

# Degree of every vertex
degrees = [G_cplx.degree(u) for u in G_cplx.iterNodes()]

# Sort by degree (descending order)
deg_count = np.array(sorted(Counter(degrees).items(), key=lambda x: x[0]))

# Plot degree distribution (log-log plot)
fig, ax = plt.subplots()
ax.set_xscale('log', base=2)
ax.set_yscale('log', base=2)
ax.loglog(deg_count[1:, 0], deg_count[1:, 1], label='Degree distribution')
_=ax.legend()

### 4.2 Clustered Random Graph Generator
Generates random graphs **with clusters**.

`n`: number of vertices in the graph

`num_clusters`: number of clusters/communities in the graph

`p_in`: intra-cluster edge probability

`p_out`: inter-cluster edge probability

In [ ]:
# Parameters
n, num_clusters, p_in, p_out = 200, 5, 0.3, 0.005

# Generate a clustered random graph
G_cluster = nk.generators.ClusteredRandomGraphGenerator(200, 5, 0.3, 0.005).generate()

# Export the graph
client.exportGraph(G_cluster)

## 5. Community Detection

### Objective
Identify the community structure of a graph.

### Methodology: Modularity Optimization
Maximize the modularity:

$$
Q = \frac{1}{2M}\sum_{i \in V}\sum_{j \in V}\left( w(i, j) - \frac{\deg(i)\deg(j)}{2M} \right)
\delta\left(c_i, c_j\right)
$$

### Popular algorithm: Louvain Method

In [ ]:
# 1. Create an instance of the Parallel Louvain Method
plm = nk.community.PLM(G_cluster)

# 2. Run the algorithm
plm.run()

# 3. Retrieve the resulting communities
communities = plm.getPartition().getVector()

# 4. Export the partitioning of the communities to Gephi
client.exportNodeValues(G_cluster, communities, "communities")